# Spark test

In [1]:
# import findspark
# findspark.init()
import sys
from random import random
from operator import add
from pyspark.sql import SparkSession as ss

spark = ss.builder.appName('PythonPi').getOrCreate()


21/10/11 21:51:42 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.27 instead (on interface en0)
21/10/11 21:51:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/11 21:51:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
partitions = 4
n = 1000000 * partitions

def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
print('Pi is roughly %f:', (4.0 * count / n))

Pi is roughly %f: 3.141688


In [3]:
spark.range(1000 * 1000 * 1000).count()

1000000000

In [ ]:
# spark = SparkSession.builder.master('local').getOrCreate()
# stack_df = spark.read.csv('sample.txt', header = True, sep = '\t', inferSchema = True)
# stack_df.filter(stack_df.name == 'vikas').show()
# stack_df.printSchema()
# print(stack_df.columns)
# stack_df.crosstab('age', 'gender').show()

# Pandas test

In [3]:
# !pip install pandas
import pandas as pd

# raw data frame
module_score = pd.DataFrame()

module_score['Module_code'] = ['INF6060', 'INF6033', 'INF6027', 'INF6029',
                               'INF6032', 'INF6050', 'INF6028', 'INF6430', 'INF6340', 'INF6000']
module_score['Module_name'] = ['Information Retrieval', 'Data and Society', 'Introduction to Data Science', 'Data Analysis',
                               'Big Data Analytics', 'Database Design', 'Data Mining and Visualisation', 'User Interface Design', 'Research Methods', 'Dissertation']
module_score['Score'] = [62, 66.4, 58, 56.4, 66, 80.2, 75, 60, 75, 80]
module_score['Weight'] = [15, 15, 15, 15, 15, 15, 15, 15, 15, 45]

module_score

/Users/Eric/.pyenv/versions/3.8.2/envs/venv-python-spark/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,Module_code,Module_name,Score,Weight
0,INF6060,Information Retrieval,62.0,15
1,INF6033,Data and Society,66.4,15
2,INF6027,Introduction to Data Science,58.0,15
3,INF6029,Data Analysis,56.4,15
4,INF6032,Big Data Analytics,66.0,15
5,INF6050,Database Design,80.2,15
6,INF6028,Data Mining and Visualisation,75.0,15
7,INF6430,User Interface Design,60.0,15
8,INF6340,Research Methods,75.0,15
9,INF6000,Dissertation,80.0,45


In [4]:
#lowercase columns
def lowercase_cols(df):
    df.columns = df.columns.str.lower()
    return df

def set_index(df, col_name = ''):
    if col_name in df.columns:
        df = df.set_index(col_name)
    return df

(module_score.pipe(lowercase_cols).pipe(set_index, 'module_code'))

,module_name,score,weight
module_code,,,
INF6060,Information Retrieval,62.0,15
INF6033,Data and Society,66.4,15
INF6027,Introduction to Data Science,58.0,15
INF6029,Data Analysis,56.4,15
INF6032,Big Data Analytics,66.0,15
INF6050,Database Design,80.2,15
INF6028,Data Mining and Visualisation,75.0,15
INF6430,User Interface Design,60.0,15
INF6340,Research Methods,75.0,15


In [5]:
credit_total = module_score['weight'].sum()

module_score['weighted_score'] = module_score.apply(
    lambda item: item['score'] * item['weight'] / credit_total, axis = 1)
module_score['weighted_score'].sum()
module_score

,module_code,module_name,score,weight,weighted_score
0,INF6060,Information Retrieval,62.0,15,5.166667
1,INF6033,Data and Society,66.4,15,5.533333
2,INF6027,Introduction to Data Science,58.0,15,4.833333
3,INF6029,Data Analysis,56.4,15,4.700000
4,INF6032,Big Data Analytics,66.0,15,5.500000
5,INF6050,Database Design,80.2,15,6.683333
6,INF6028,Data Mining and Visualisation,75.0,15,6.250000
7,INF6430,User Interface Design,60.0,15,5.000000
8,INF6340,Research Methods,75.0,15,6.250000
9,INF6000,Dissertation,80.0,45,20.000000


# Recap Pyspark

In [6]:
from pyspark.sql import SparkSession as ss

spark = ss.builder.master('local').getOrCreate()


In [7]:
df = spark.createDataFrame(module_score).persist()
# df.describe()
# df.printSchema()
# df.show()

df = df.withColumn('weighted_score', df['weighted_score'].cast('integer'))
df.orderBy('weighted_score', ascending=0).show(3) # return as dataframe

df.collect()[0:3] # return as row type and query top 3 items

+-----------+----------------+-----+------+--------------+
|module_code|     module_name|score|weight|weighted_score|
+-----------+----------------+-----+------+--------------+
|    INF6000|    Dissertation| 80.0|    45|            20|
|    INF6050| Database Design| 80.2|    15|             6|
|    INF6340|Research Methods| 75.0|    15|             6|
+-----------+----------------+-----+------+--------------+
only showing top 3 rows



[Row(module_code='INF6060', module_name='Information Retrieval', score=62.0, weight=15, weighted_score=5),
 Row(module_code='INF6033', module_name='Data and Society', score=66.4, weight=15, weighted_score=5),
 Row(module_code='INF6027', module_name='Introduction to Data Science', score=58.0, weight=15, weighted_score=4)]